<a href="https://colab.research.google.com/github/SamT2023/Flooding/blob/main/download_from_website.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download extract from the website

In [ ]:

import os
import numpy as np
import pandas as pd
import xarray as xr
import tarfile
import patoolib
from osgeo import gdal
import rasterio
import rasterio.mask
import geopandas as gpd
import scipy.sparse as sparse
import fiona
import scipy
import rioxarray as rxr
from shapely.geometry import mapping
import netCDF4
import requests



####DOWNLOAD CHIRPS DATA FROM A WEBSITE
os.chdir(r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\CHIRPS_zip")  ####Folder to store zipped/downloaded files

###### watch the following for the details https://www.youtube.com/watch?v=XGUS6DYZfCc ############

years = np.arange(1981,2022)
months = ['01','02','03','04','05','06','07','08','09','10','11','12']

for year in years:
    for mon in months:
        url = 'https://data.chc.ucsb.edu/products/CHIRPS-2.0/africa_monthly/tifs/chirps-v2.0.' + str(year)+'.'+ mon + '.tif.gz'
        r = requests.get(url, allow_redirects = True)
        open(str(year)+'_'+mon+'.gz', 'wb').write(r.content)

###unzip downloaded tiff files
output_dir = r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\CHIRPS_Unzipped" #your unzipped files will be stored here

for zipfiles in os.listdir(r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\CHIRPS_zip"):  ###Folder having zipped files
    os.chdir(r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\CHIRPS_zip")  #####Folder having zipped files

    if zipfiles[-3:] == '.gz':
        patoolib.extract_archive(zipfiles, outdir=output_dir)

# List all files in the output directory and rename them to have the .tif extension
for filename in os.listdir(output_dir):
    base, ext = os.path.splitext(filename)
    if ext.lower() not in ['.tif', '.tiff']:
        os.rename(os.path.join(output_dir, filename), os.path.join(output_dir, base + '.tif'))

### Clips the tif files using the input shapefile
files = os.listdir(r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\CHIRPS_Unzipped")  ###extracted tif files location

shapefile_ET = gpd.read_file(r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\Lake_Victoria_Basin_Shapefile\LV_Basin.shp")  ##Basin shp

for f in files:
    #f=files[0]
    if f[-4:]=='.tif':

        #imagery = rasterio.open(f)

        with fiona.open(r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\Lake_Victoria_Basin_Shapefile\LV_Basin.shp",'r') as shapefile:  ##Basin shp
            for feature in shapefile:
                shapes = [feature['geometry']]

        os.chdir(r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\CHIRPS_Unzipped")  ######extracted tif files location

        with rasterio.open(f) as src:
           out_image,out_transform = rasterio.mask.mask(src,shapes,crop=True)
           out_meta = src.meta

        out_meta.update({
            "driver":"Gtiff",
            "height": out_image.shape[1],
            "width": out_image.shape[2],
            'transform':out_transform
        })

        os.chdir(r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\Clipped_files")   ###Folder to save output clipped tif files

        with rasterio.open(f, "w", **out_meta) as dest:
            dest.write(out_image)


####################### convert each tif files of the study area to netcdf file ###########################

input_folder = r"E:\Clipped_files"
output_folder = r"E:\Netcdf files"

os.chdir(input_folder)
files = os.listdir(input_folder)


for f in files:
    if f[-4:]=='.tif':
        output = os.path.join(output_folder, f[12:16] + "_" + f[17:19] + '.nc')
        input_path = os.path.join(input_folder, f)

        ds = gdal.Translate(output, input_path, format='NetCDF')


        print(f"Converted {f} to {output}")

################# now lets combine them using xarray  #####################
# Specify the folder path containing the NetCDF files

def combine_netCDF_files(input_folder, output_folder, output_filename):
    os.makedirs(output_folder, exist_ok=True)

    os.chdir(input_folder)
    files = os.listdir(input_folder)

    # List to store individual NetCDF datasets
    datasets = []

    for f in files:
        if f.endswith('.nc'):
            file_path = os.path.join(input_folder, f)
            dataset = xr.open_dataset(file_path)
            datasets.append(dataset)

    # Combine all datasets along the time dimension
    combined_dataset = xr.concat(datasets, dim='time')

    # Save the combined dataset to a new NetCDF file
    output_combined_file = os.path.join(output_folder, output_filename)
    combined_dataset.to_netcdf(output_combined_file)

    print("Combined NetCDF file saved to:", output_combined_file)

# Example usage:
input_folder = r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\Netcdf files"
output_folder = r"C:\Users\samto\OneDrive\Desktop\Lake_victoria_shape_ file\Final_nc"
output_filename = "LVB_chirps.nc"

combine_netCDF_files(input_folder, output_folder, output_filename)
print("NetCDF files combined successfully!")